# Webscraper to get Canadian postal code data 

In [126]:
import pandas as pd

There exist multiple ways to obtain data from the web when API's are not directly available.  
I will use pandas awesome method read_html to get the table I want.

First, I read all tables available within the Wikipedia link and chose the first one (pandas provides a list of dataframes), headers included.  


In [127]:
dfs = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)
#I find the first table by printing 
#for df in dfs:
#    print(df)
df = dfs[0]
df.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True)
#Count how many postalcodes exist per Borough
df.groupby('Borough').count()['PostalCode']

Borough
Central Toronto     17
Downtown Toronto    37
East Toronto         7
East York            6
Etobicoke           45
Mississauga          1
North York          38
Not assigned        77
Queen's Park         1
Scarborough         37
West Toronto        13
York                 9
Name: PostalCode, dtype: int64

77 entries are not assigned and hence will be dropped

In [128]:
df = df[df['Borough']!='Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Now Postalcodes that contain multiple neighborhoods will be summarized and all neighborhoods are concatinated with comma seperation

In [129]:
df_new = df.groupby('PostalCode')['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index()

This code below would do the same job, I played around with it but found the above one easier to read

In [130]:
#Alternative code with same results
#df_new = (df['Neighborhood'].str.split(', ')
#                    .groupby(df['PostalCode'])
#                    .agg(lambda x: ', '.join(set(y for z in x for y in z)))
#                    .reset_index())
#df_new.shape

Since the previous operation dropped the Borough column it will be merged  from the previous data frame

In [131]:
df_new = pd.merge(df_new,df[['PostalCode','Borough']],on='PostalCode', how='left')
df_new = df_new.drop_duplicates()

M7A contains neighborhood with value 'not assigned'. It will be replaced by its Borough

In [132]:
df_new.loc[df_new['Neighborhood'] == 'Not assigned']

,PostalCode,Neighborhood,Borough
159,M7A,Not assigned,Queen's Park


Last, replace  all Neighborhoods that are not assigned with their respective borough

In [133]:
df_new['Neighborhood'].where(df_new['Neighborhood']!='Not assigned', other=df_new['Borough'], inplace=True)
#df_new['Neighborhood'] = df_new['Borough'].where(df_new['Neighborhood'] == 'Not assigned')
#if error , ''

In [134]:
df_new.loc[df_new['Neighborhood'] == "Queen's Park"]

,PostalCode,Neighborhood,Borough
159,M7A,Queen's Park,Queen's Park


In [135]:
df_new.shape

(103, 3)